In [ ]:
import os
import json
import time
import re
import pandas as pd
import numpy as np
from datetime import datetime
import glob

from src.util import prnt
from src.util import print_all_output
from src.settings import *

## Inspect Output Files

In [ ]:

folder_path = "output/"
prefix = "edm26-SS2025-etherpad-07"

for file in sorted(glob.glob(os.path.join(folder_path, f"{prefix}*.csv"))):
    print(f"\n{'='*60}")
    print(f"Datei: {os.path.basename(file)}")

    df = pd.read_csv(file)
    
    print(f"Column names ({len(df.columns)}):")
    print(df.columns.tolist())
    print(df.head(1))

### TODO

* 

In [ ]:
#tmp_df = pd.read_csv('output/lak25-WS2024_25-etherpad-01-data-chats.csv')
#tmp_df = pd.read_csv('output/lak25-WS2024_25-etherpad-01-data-scrolls.csv')
#tmp_df = pd.read_csv('output/lak25-WS2024_25-etherpad-01-data-comments.csv')
#tmp_df = pd.read_csv('output/lak25-WS2024_25-etherpad-01-data-comment-replies.csv')
#tmp_df = pd.read_csv('output/lak25-WS2024_25-etherpad-01-data-textedits.csv')
tmp_df = pd.read_csv('output/lak25-WS2024_25-etherpad-02-textchanges.csv')
#tmp_df
#len(pd.unique(tmp_df['moodle_group_id']))
len(pd.unique(tmp_df['moodle_user_id']))
#tmp_df.shape


# Samples sizes

In [ ]:
import pandas as pd

# Define semesters and corresponding data dumps
semesters = [
    #('WS2021_22', 'dump20240826'),
    #('SS2022', 'dump20240826'),
    #('WS2022_23', 'dump20240826'),
    #('SS2023', 'dump20240826'),
    #('WS2023_24', 'dump20240826'),
    #('SS2024', 'dump20240826'),
    ('WS2024_25', 'dump20241209'),
]

# Initialize total counters
total_users = 0
total_groups = 0
project_name = 'lak25'

# Store results for LaTeX table
results = []

for the_semester in semesters:
    semester = the_semester[0]
    the_dump = the_semester[1]
    path = os.path.join(output_path, f'{project_name}{semester}-etherpad-06-author_relations_summary.csv')
    print(path)
    # Read data
    user_df = pd.read_csv(path)

    # Count unique users and groups
    num_users = len(user_df['moodle_user_id'].unique())
    num_groups = len(user_df['moodle_group_id'].unique())

    # Calculate group sizes (authors per group)
    group_sizes = user_df.groupby('moodle_group_id')['moodle_user_id'].nunique()
    min_group_size = group_sizes.min()
    max_group_size = group_sizes.max()
    mean_group_size = group_sizes.mean()

    # Print progress
    print(f'Processing {semester}: Groups={num_groups}, Users={num_users}')
    print(f'  Group sizes: Min={min_group_size}, Max={max_group_size}, Mean={mean_group_size:.2f}')
    
    # Accumulate totals
    total_users += num_users
    total_groups += num_groups

    # Append results for LaTeX
    results.append((semester, num_groups, num_users, min_group_size, max_group_size, mean_group_size))

# Generate LaTeX table
latex_table = "\\begin{table}[h]\n\\centering\n\\begin{tabular}{|l|r|r|r|r|r|}\n\\hline\n"
latex_table += "Semester & Groups & Authors & Min Members & Max Members & Mean Members \\\\\n\\hline\n"

for row in results:
    latex_table += f"{row[0]} & {row[1]} & {row[2]} & {row[3]} & {row[4]} & {row[5]:.2f} \\\\\n"

latex_table += "\\hline\n"
latex_table += f"Total & {total_groups} & {total_users} & - & - & - \\\\\n"
latex_table += "\\hline\n\\end{tabular}\n\\caption{Summary of group sizes and author counts per semester}\n\\label{tab:group_summary}\n\\end{table}"

# Print LaTeX table
print("\nLaTeX Table:\n")
print(latex_table)

In [ ]:
import pandas as pd
semester = "WS2024_25"
df_raw_text = pd.read_csv(f'{output_path}{project_name}{semester}-02.1-text-revisions.csv')
#df_raw_text = pd.read_csv(f'{output_path}{project_name}{semester}-02.1-text-quality-new.csv')
df_raw_text.columns = [
    "group_id", 
    "pad_id",
    "timestamp", 
    "current_time_threshold",
    "text",
]
print(
    len(df_raw_text['group_id'].unique()),
    len(df_raw_text['timestamp'].unique()),
    #df_raw_text.text.str.len() / df_raw_text.shape[0],
    df_raw_text.shape[0]
)
len(df_raw_text.loc[10, 'text'])

df_raw_text[df_raw_text['group_id']==3947].shape


In [ ]:
import spacy
import textdescriptives as td
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("de_core_news_md")
nlp.add_pipe("textdescriptives/pos_proportions")
#doc = nlp("The world is changed. I feel it in the water. I feel it in the earth. I smell it in the air. Much that once was is lost, for none now live who remember it.")
doc = nlp("Der Müller geht schön.")
# all attributes are stored as a dict in the ._.pos_proportions attribute
doc._.pos_proportions

# extract to dataframe
td.extract_df(doc)